In [10]:
import subprocess
import os
import pandas as pd
import numpy as np
import xarray as xr

from Bio import SeqIO

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler
import torch.utils.data as data

import pickle
import random

In [7]:
# I tried several data sets but in the end chose to use only the two mouse genomes pulled in below.
# fasta_path = './Danio_rerio.GRCz11.dna.chromosome.10.extrasmall.fa'

# fasta_sequences = SeqIO.parse(open(fasta_path),'fasta')
# fasta_path = './Danio_rerio.GRCz11.dna_sm.chromosome.22.fa'

# fasta_sequences = SeqIO.parse(open(fasta_path),'fasta')

In [252]:
fasta_path = './data/Mus_musculus.GRCm39.dna.chromosome.19.large.fa'

mm19_sequences = SeqIO.parse(open(fasta_path),'fasta')

In [253]:
fasta_path = './data/Mus_musculus.GRCm39.dna.chromosome.17.fa'

mm17_sequences = SeqIO.parse(open(fasta_path),'fasta')

In [250]:
sequences = {}

In [254]:
for f in mm17_sequences:
    name, sequence = f.id, str(f.seq)
    sequences['mm_17'] = sequence
for f in mm19_sequences:
    name, sequence = f.id, str(f.seq)
    sequences['mm_19'] = sequence

In [255]:
sequences.keys()

dict_keys(['mm_17', 'mm_19'])

In [117]:
ltr_finder_dir = os.system("cd ./LTR_finder/source")
path = './LTR_Finder/source/ltr_finder'
fasta_path = './data/Mus_musculus.GRCm39.dna.chromosome.17.fa'
f = open("mus_musculus_17.txt", "w")
subprocess.run([path, fasta_path, '> file.txt'], stdout=f)
f.close

<function TextIOWrapper.close()>

In [29]:
ltr_finder_dir = os.system("cd ./LTR_finder/source")
path = './LTR_Finder/source/ltr_finder'
fasta_path = './data/Mus_musculus.GRCm39.dna.chromosome.19.large.fa'
f = open("mus_musculus_19_large.txt", "w")
subprocess.run([path, fasta_path, '> file.txt'], stdout=f)
f.close

<function TextIOWrapper.close()>

In [256]:
# reads a line from the ltr_finder output. Returns start/end position of TE
def parse_ltr_loc(line):
    first_num_start = 11
    first_num_end = line.find(' ', first_num_start, -1)
    second_num_start = first_num_end + 3
    second_num_end = line.find(' ', second_num_start, -1)

    ltr_start_loc = int(line[first_num_start:first_num_end])
    ltr_end_loc = int(line[second_num_start:second_num_end])
    
    return (ltr_start_loc, ltr_end_loc)

In [199]:
# Removes duplicates from list of detected ltr positions
def clear_dups(ltr_locations):
    starts = []
    locs = []
    for pair in ltr_locations:
        if pair[0] not in starts:
            starts.append(pair[0])
            locs.append(pair)

    return locs

In [ ]:
ltrs = {}

In [257]:
# Parse the LTR_Finder outputs to get a list of LTR start/end positions
# Importnat: Keys represent the sequence the TEs belong to
with open('data/mus_musculus_19_large.txt', 'r') as f:
    lines = f.readlines()
    local = []
    for i, line in enumerate(lines):
        if 'Location : ' in line:
            local.append(parse_ltr_loc(line))
    ltrs['mm_19'] = clear_dups(local)

with open('data/mus_musculus_17.txt', 'r') as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        if 'Location : ' in line:
            local.append(parse_ltr_loc(line))
    ltrs['mm_17'] = clear_dups(local)

In [137]:
np_loc = np.asarray(locs)

In [268]:
# Some stats on the TEs found (The current output might not represent the data actually used in the paper)
maxl = np.max([l[1] - l[0] for l in np_loc])
minl = np.min([l[1] - l[0] for l in np_loc])
avgl = np.mean([l[1] - l[0] for l in np_loc])
medl = np.median([l[1] - l[0] for l in np_loc])
print(len(locs))
print(maxl)
print(minl)
print(maxl - minl)
print(avgl)
print(medl)

30
19673
1515
18158
6532.866666666667
5401.0


In [201]:
ltrs

{'mm_19': [(33297, 38704),
  (47802, 53318),
  (149758, 153662),
  (161390, 167784),
  (223723, 228816),
  (1061401, 1066389),
  (1639290, 1658963),
  (2063477, 2078101),
  (2403074, 2419209),
  (2576470, 2578356),
  (2909568, 2916137),
  (3591011, 3594175),
  (3962553, 3981691),
  (4990900, 4997335),
  (5532154, 5543638),
  (5654710, 5657524),
  (5673222, 5678617),
  (6210472, 6214224),
  (7682640, 7689211),
  (8664442, 8671502),
  (8944706, 8946601),
  (9801152, 9809135),
  (10201908, 10208303),
  (10358534, 10360049),
  (10514781, 10520176),
  (10700264, 10702149),
  (11372458, 11374892),
  (12527384, 12529260),
  (13612985, 13621711),
  (13892767, 13894647)],
 'mm_17': [(33297, 38704),
  (47802, 53318),
  (149758, 153662),
  (161390, 167784),
  (223723, 228816),
  (1061401, 1066389),
  (1639290, 1658963),
  (2063477, 2078101),
  (2403074, 2419209),
  (2576470, 2578356),
  (2909568, 2916137),
  (3591011, 3594175),
  (3962553, 3981691),
  (4990900, 4997335),
  (5532154, 5543638),
  (

In [130]:
# Set the size of sequence chunks (for inputs and target)
# The size should be bigger than the average TE length.
size = 15000

In [131]:
# Creates one-hot encoding input given a sequence and chunk size.
def get_input_channels(sequence, size):
  a = np.zeros(size)
  g = np.zeros(size)
  c = np.zeros(size)
  t = np.zeros(size)
  for i, chr in enumerate(sequence):
    if chr == 'A':
      a[i] = 1
    if chr == 'G':
      g[i] = 1
    if chr == 'C':
      c[i] = 1
    if chr == 'T':
      t[i] = 1

  return [a, g, c, t]    

In [258]:
x = []
masks = []

In [259]:
for i in range(4): #force a larger sample size. Not ideal but okay for now.
    
    # For each sequence collected above
    for chromosome in sequences.keys():
        locs = ltrs[chromosome]
        seq = sequences[chromosome]
        
        # For each LTR location within this chromosome, create an input and matching target
        for loc in locs:
            ltr_length = loc[1] - loc[0] + 1
            remaining_sequence = size - ltr_length
            
            # If LTR length is larger than the chunk size, choose the starting location randomly
            if remaining_sequence < 1:
                chunk = int((loc[1] - loc[0]) * 0.1)
                padding = random.randrange(-chunk, chunk)
                start = loc[0] + chunk
                end = start + size

                inpt = get_input_channels(seq[start:end], size)
                mask = np.ones(size)
            else: # Otherwise, choose a random start and end position that will contain the current LTR
                padding_left = random.randrange(0, remaining_sequence)
                padding_right = remaining_sequence - padding_left

                start = loc[0] - padding_left
                end = loc[1] + padding_right + 1

                inpt = get_input_channels(seq[start:end], size)

                mask = np.zeros(size)
                mask[padding_left+1:padding_left+ltr_length+1] = 1

            x.append(inpt)
            masks.append(mask)

In [261]:
data = {'x': x, 'masks': masks}

In [237]:
# More tests
x = np.asarray(x)
masks = np.asarray(masks)

In [238]:
# What percentage of data is actually made of TEs. Should be around half.
np.sum(masks) / (masks.shape[0]*size)

0.4035906210392902

In [ ]:
file = open('./data/te_data.pkl', 'wb')

pickle.dump(data, file)

file.close()